In [1]:
#Importação de Bibliotecas
import pandas as pd
import requests
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

In [2]:
#Função para criar Token de acesso de um usuário em específico: create_user_access_token()

def create_user_access_token(username,clientID,secret):
    '''Função que utiliza a biblioteca Spotipy para realizar a criação de
       um token específico para coleta de dados pessoais do usuário
    '''
    # Configurações do aplicativo Spotify
    CLIENT_ID = clientID
    CLIENT_SECRET = secret
    REDIRECT_URI = 'http://localhost:8888/callback' 
    USERNAME = username  

    # Escopo de permissões que você deseja obter
    SCOPE = 'user-read-recently-played user-read-private user-read-email user-top-read'  # Solicita acesso ao histórico de reprodução recente do usuário

    # Autorização do usuário
    token = util.prompt_for_user_token(username=USERNAME,
                                       scope=SCOPE,
                                       client_id=CLIENT_ID,
                                       client_secret=CLIENT_SECRET,
                                       redirect_uri=REDIRECT_URI)

    return 'Bearer '+token

In [3]:
#Função para coletar os dados mais recentes de execução

def query_recent(token):
    '''Função que realiza a consulta dos últimos registros do player do spotify.
       Esta função executa a create_user_access_token(), pois necessita de um escopo específico.
       O limite padrão é 150, mas pode ter como entrada outro valor.
    '''
    headers = {
        "Authorization":  token
            }

    # Definir URL para pesquisa
    recently_played_url = "https://api.spotify.com/v1/me/player/recently-played"

    # Definindo parâmetro de limite de resultado. Inicialmente definido para 50. Usar com parcimônia, pois o Spotify bloqueia consultas muito longas
    params = {
        "limit": 50
    }

    # Realizar o Request na API do Spotify
    response = requests.get(recently_played_url, headers=headers, params=params)
    recently_played_data = response.json()

    # Extrair dados da resposta
    tracks = []
    for item in recently_played_data.get("items", []):
        track = item.get("track")
        if track:
            track_info = {
                "Nome da Música": track.get("name"),
                "Nome do Artista": track.get("artists")[0].get("name"),
                "Nome do Álbum": track.get("album").get("name"),
                "Link da Música": track.get("uri"),
                "Data de Execução": item.get("played_at")
            }
            tracks.append(track_info)
    #Retorno da Função
    return tracks

In [4]:
#Criação de Token do Spotify.
#O Token tem validade de 60 minutos
#Username pode ser coletado diretamente do painel de gerenciamento da conta do usuário
#O Client ID e o Secret precisam ser criadas no Dashboard de API do Spotify
token = create_user_access_token('22rxsto4zfjdqqyfdhx27v3jq','7d486b31ccdc419798ea333e9aa9c91c','018d8dfeb158496b80452b6a4ccbe28d')

In [5]:
#Criando Dataframe com o resultado do request à API do Spotify
ultimas_musicas = pd.DataFrame.from_dict(query_recent(token))
ultimas_musicas

,Nome da Música,Nome do Artista,Nome do Álbum,Link da Música,Data de Execução
0,Koroba,Tiwa Savage,Celia,spotify:track:18t2ofQiViidZHxOBVjpD1,2023-10-07T01:12:49.850Z
1,In Fact (feat. Gabzy),melvitto,THENIGHTISYOUNG.,spotify:track:2rFPUq8a7gvO0IuaN9LpV7,2023-10-07T01:10:15.246Z
2,Love Language,Juls,Sounds of My World,spotify:track:7Jqa7kWKN4d2Fn7FESctLz,2023-10-07T01:06:29.049Z
3,Muse,Asake,Mr. Money With The Vibe,spotify:track:3Nx9nMHMFgkeF8xpVk1Ev0,2023-10-07T01:03:05.207Z
4,Mayana,Aṣa,V,spotify:track:5WeLc7gPjNabQ1AMk5lxrl,2023-10-07T01:01:06.497Z
5,Testimony,Bnxn,Testimony,spotify:track:5L845uQOeyOSSAhGPFEmLc,2023-10-07T00:58:28.130Z
6,Born Again,Adekunle Gold,Catch Me If You Can,spotify:track:2XhX2hzgu66MKw09tbbwCE,2023-10-07T00:56:01.374Z
7,Fever,Soundz,Fever,spotify:track:3GAu2fSg9deV207noxMKl0,2023-10-07T00:54:09.228Z
8,Yebo/Sema,Masego,Studying Abroad: Extended Stay,spotify:track:0ed7g01Suk6ITt0vwT2BgD,2023-10-07T00:51:02.551Z
9,Love Somebody,Rotimi,All or Nothing (Deluxe),spotify:track:2VR1nIY2M4W2AfNOg4uFR7,2023-10-07T00:47:12.897Z
